## Scrapping data from flipkart

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


ratings=[]
reviews=[]
ter=[]
rev=[]
ab=[]

for page in range(1,1000):
    
    
    url = 'https://www.flipkart.com/poco-m3-power-black-64-gb/product-reviews/itmb49cc10841be2?pid=MOBFZTCUTAYPJHHR&lid=LSTMOBFZTCUTAYPJHHR2ZVC1N&marketplace=FLIPKART'
    header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    }
    query_url = url + '&page={}'.format(page)
    r = requests.get(query_url, headers = header)
    text = r.text
    soup = BeautifulSoup(text, 'html.parser')
    try:
                
        results_page = soup.find_all('div',{'class':'_1YokD2 _3Mn1Gg col-9-12'})[0]
        ab=[i.text for i in results_page.find_all('div',{'class':'_1AtVbE col-12-12'})]
        ab=ab[2:-1]
        ter=[None]*10
        for u in range(0,10):
            ter[u]=ab[u][1:]
        rev+=ter
        for i in ab:
            ratings+=i[0]
           
    except IndexError:
        pass

for y in range(0,len(rev)):
    yu=[str(rev[y]).split('READ MORE')[0]]
    reviews+=yu     
df=pd.DataFrame({'ratings':ratings,'reviews':reviews})
df.to_csv('flipkart_phone.csv')

## using the created csv file to perform natural language processing

In [147]:
data=pd.read_csv('flipkart_phone.csv')

In [148]:
data.head()

,Unnamed: 0,ratings,reviews
0,0,5,Highly recommendedGreat Phone at this Price po...
1,1,4,Really NiceGood mobile poco m3Pros: Fullhd dis...
2,2,5,WonderfulGood phone battery🔋 And camera This p...
3,3,5,"TerrificOne word review "" Value for Money""It h..."
4,4,4,Worth the moneySound quality super 3 type soun...


In [149]:
data.drop(["Unnamed: 0"],axis=1,inplace=True)

In [150]:
data

,ratings,reviews
0,5,Highly recommendedGreat Phone at this Price po...
1,4,Really NiceGood mobile poco m3Pros: Fullhd dis...
2,5,WonderfulGood phone battery🔋 And camera This p...
3,5,"TerrificOne word review "" Value for Money""It h..."
4,4,Worth the moneySound quality super 3 type soun...
...,...,...
7185,5,Best in the market!Super
7186,3,FairBattery at it's best for this pricePerform...
7187,4,Very GoodGood mobile
7188,5,Highly recommendedGood mobile poco m3


In [151]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
import nltk
from string import punctuation
from nltk.corpus import stopwords,wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer

lemma=WordNetLemmatizer()
stemmer=PorterStemmer()

ext=["i've","i'll","we'll","'ve'","it's"]


def text_process(t):
    all_words=[]
    t=t.replace('.','. ').replace('!','! ').replace('?','? ')
    for sent in nltk.sent_tokenize(t.lower()):
        words=nltk.word_tokenize(sent)
        words=[word for word in words if(word not in stopwords.words('english')) and (word not in ext)]
        words=[lemma.lemmatize(word, wordnet.ADJ) for word in words]
        words=[stemmer.stem(word) for word in words]
        words=[word for word in words if word not in punctuation]
        words=[word for word in words if not word.isnumeric()]
        all_words+=words
    all_words=''.join([ch for ch in ' '.join(all_words) if (ch not in punctuation) and (not ch.isnumeric())])
    return all_words


tfidfv=TfidfVectorizer()
tf_idf=tfidfv.fit_transform(data['reviews'].apply(text_process))
tfdf=pd.DataFrame(tf_idf.toarray(),columns=tfidfv.get_feature_names())


In [152]:
tfdf.describe()

,aa,ab,abd,abest,abl,ablut,absolut,absorv,abt,abu,...,𝓹𝓻𝓸𝓭𝓾𝓬𝓽,𝓻𝓮𝓼𝓹𝓮𝓬𝓽,𝓽𝓸,𝘣𝘶𝘵,𝘤𝘢𝘮𝘦𝘳𝘢,𝘪𝘮𝘱𝘳𝘰𝘷𝘦,𝘪𝘵,𝘲𝘶𝘢𝘭𝘪𝘵𝘺,𝚏𝚘𝚛,𝚗𝚒𝚌𝚎
count,7190.000000,7190.000000,7190.000000,7190.000000,7190.000000,7190.000000,7190.000000,7190.000000,7190.000000,7190.000000,...,7190.000000,7190.000000,7190.000000,7190.000000,7190.000000,7190.000000,7190.000000,7190.000000,7190.000000,7190.000000
mean,0.000121,0.000043,0.000038,0.000086,0.000600,0.000074,0.003077,0.000039,0.000069,0.000080,...,0.000053,0.000053,0.000053,0.000110,0.000110,0.000110,0.000110,0.000110,0.000065,0.000131
std,0.007333,0.003660,0.003241,0.007308,0.013633,0.004439,0.034802,0.003317,0.005892,0.006759,...,0.004457,0.004457,0.004457,0.006576,0.006576,0.006576,0.006576,0.006576,0.005522,0.011072
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.488826,0.310341,0.274812,0.619704,0.405965,0.266146,0.669211,0.281284,0.499623,0.573135,...,0.377964,0.377964,0.377964,0.394322,0.394322,0.394322,0.394322,0.394322,0.468192,0.938873


In [153]:
tfdf

,aa,ab,abd,abest,abl,ablut,absolut,absorv,abt,abu,...,𝓹𝓻𝓸𝓭𝓾𝓬𝓽,𝓻𝓮𝓼𝓹𝓮𝓬𝓽,𝓽𝓸,𝘣𝘶𝘵,𝘤𝘢𝘮𝘦𝘳𝘢,𝘪𝘮𝘱𝘳𝘰𝘷𝘦,𝘪𝘵,𝘲𝘶𝘢𝘭𝘪𝘵𝘺,𝚏𝚘𝚛,𝚗𝚒𝚌𝚎
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7186,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7187,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7188,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### creating a machine learning model to predict ratings based on reviews

In [154]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.svm import SVC


xtrain,xtest,ytrain,ytest=train_test_split(tfdf,data['ratings'],test_size=0.25,random_state=2)

model_bnb=BernoulliNB().fit(xtrain,ytrain)
print(round(model_bnb.score(xtest,ytest)*100,2))

model_mnb = MultinomialNB().fit(xtrain,ytrain)
print(round(model_mnb.score(xtest,ytest)*100,2))

model_svc = SVC().fit(xtrain,ytrain)
print(round(model_svc.score(xtest,ytest)*100,2))

67.19
74.64
86.1


#### We received an accuracy of 67.19% from Bernoulli naive bayes, 74.64% from Multinomial naive bayes and reached heighest accuracy from support vector classifier of 86.1%.

#### To improve its accuracy we will further explore support vector classifier

In [159]:
from sklearn.model_selection import GridSearchCV

params = {'C':[0.1,1,10], 'gamma':[0.1,1,10], 'degree':[1,2], 'kernel':['linear', 'poly', 'rbf', 'sigmoid']}

model_grid = GridSearchCV(SVC(),param_grid=params,verbose=3,cv=2).fit(xtrain,ytrain)

Fitting 2 folds for each of 72 candidates, totalling 144 fits
[CV 1/2] END ......C=0.1, degree=1, gamma=0.1, kernel=linear; total time=  34.2s
[CV 2/2] END ......C=0.1, degree=1, gamma=0.1, kernel=linear; total time=  34.2s
[CV 1/2] END ........C=0.1, degree=1, gamma=0.1, kernel=poly; total time=  35.7s
[CV 2/2] END ........C=0.1, degree=1, gamma=0.1, kernel=poly; total time=  35.1s
[CV 1/2] END .........C=0.1, degree=1, gamma=0.1, kernel=rbf; total time=  50.2s
[CV 2/2] END .........C=0.1, degree=1, gamma=0.1, kernel=rbf; total time=  49.5s
[CV 1/2] END .....C=0.1, degree=1, gamma=0.1, kernel=sigmoid; total time=  35.7s
[CV 2/2] END .....C=0.1, degree=1, gamma=0.1, kernel=sigmoid; total time=  35.3s
[CV 1/2] END ........C=0.1, degree=1, gamma=1, kernel=linear; total time=  34.3s
[CV 2/2] END ........C=0.1, degree=1, gamma=1, kernel=linear; total time=  34.3s
[CV 1/2] END ..........C=0.1, degree=1, gamma=1, kernel=poly; total time=  34.3s
[CV 2/2] END ..........C=0.1, degree=1, gamma=1

In [161]:
print(model_grid.best_estimator_)

SVC(C=10, degree=1, gamma=0.1)


In [170]:
mos=model_grid.best_estimator_.fit(xtrain,ytrain)

In [177]:
print(round(mos.score(xtest,ytest)*100,2))

86.87


#### i reached an accuracy 86.87%. Accuracy can be improved by increasing the parameters provided in params. Due to my system's limitations i am performing with limited parameters.